In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

### 1. Carga de Datos
Para este ejemplo, vamos a simular datos de una serie temporal.

In [ ]:
np.random.seed(42)
date_rng = pd.date_range(start='1/1/2020', end='31/12/2023', freq='D')
df = pd.DataFrame(date_rng, columns=['date'])
df['var1'] = np.random.randn(len(date_rng)) * 10
df['var2'] = np.random.randn(len(date_rng)) * 20
df['var3'] = np.random.randn(len(date_rng)) * 30
df['var4'] = np.random.randn(len(date_rng)) * 40
df['var5'] = np.random.randn(len(date_rng)) * 50
df['var6'] = np.random.randn(len(date_rng)) * 60
df.set_index('date', inplace=True)

### 2. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Visualizamos las primeras filas del DataFrame
print("Primeras filas del DataFrame:")
print(df.head())

# Descripción estadística de los datos
print("\nDescripción estadística:")
print(df.describe())

# Visualización de las series temporales
plt.figure(figsize=(15, 10))
for i, col in enumerate(df.columns):
    plt.subplot(3, 2, i+1)
    plt.plot(df[col])
    plt.title(col)
plt.tight_layout()
plt.show()

### 3. Descomposición de la Serie Temporal

In [ ]:
# Descomponemos la serie temporal para analizar tendencia, estacionalidad y residuales
result = seasonal_decompose(df['var1'], model='additive', period=365)
result.plot()
plt.show()

### 4. Preprocesamiento de Datos

In [ ]:
# Normalizamos los datos para el modelo de predicción
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df)

# Convertimos el array normalizado de nuevo a DataFrame
scaled_df = pd.DataFrame(scaled_df, columns=df.columns, index=df.index)

# División en conjunto de entrenamiento y prueba
train_size = int(len(scaled_df) * 0.8)
train_df, test_df = scaled_df[:train_size], scaled_df[train_size:]

### 5. Creación de un Modelo de Predicción
Para simplificar, usaremos un modelo de regresión lineal. Nuestro objetivo será predecir 'var1' en función de las otras variables.

In [ ]:
# División en variables independientes (X) y dependiente (y)
X_train, y_train = train_df.drop('var1', axis=1), train_df['var1']
X_test, y_test = test_df.drop('var1', axis=1), test_df['var1']

# Creación y entrenamiento del modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predicción
y_pred = model.predict(X_test)

### 6. Evaluación del Modelo

In [ ]:
# Calculamos el error cuadrático medio (MSE) y el coeficiente de determinación (R²)
mse = mean_squared_error(y_test, y_pred)
r2 = model.score(X_test, y_test)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")

# Visualización de la predicción
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('Actual vs Predicted')
plt.show()